In [123]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

# Configurando a exibição do Pandas Data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
# pd.options.display.float_format = '{:.2f}'.format

In [124]:
df = pd.read_csv('producers.csv',
                 sep=',',
                 encoding='utf-8',
                 na_values=['nan']
                 )

df_racoes = pd.read_csv('feed.csv',
                        sep=';',
                        encoding='ISO-8859-1',
                        na_values=['nan']
                        )

df_rotas_kml = pd.read_csv('routes.csv',
                           sep=',',
                           encoding='utf-8')

df = df.rename(columns={'COD_PROD': 'cod_prod'})

df_rotas_kml = df_rotas_kml[['cod_prod', 'route_id', 'Distância (km)']]

df = pd.merge(df, df_rotas_kml, on='cod_prod', how='left')

df = df[df['Status do lote'] == 'ativo']

drop_raw = ['Empresa', 'Data carregamento', 'coughs', 'Status SmartMIC', 'CustoRação esperado',	'CustoRação realizado',	'CustoMed esperado',	'CustoMed realizado',
            'Investimento previsto',	'Investimento real',	'InvestMed previsto',	'InvestMed real',	'InvestGlobal previsto',	'InvestGlobal real', 'Mortalidade',
            'Mortalidade%', 'Consumo acc',	'Consumo esperado',	'CA acc',	'CA esperada',	'GPDacc',	'GPD esperado',	'VARPeso',	'VARConsumo',	'VARCA',	'VARGPD',	'ClassPeso',
            'ClassConsumo',	'ClassCA',	'ClassGPD',	'DelEvolução',	'DelCarregamentoPS',	'DelCarregamento esperado',	'DiffCarregamento',	'PeriodoGlobal esperado',
            'PeriodoGlobal realizado', 'Resultado',	'Resultado cabeça',	'Resultado kg',	'Efeito VAR-CA', 'Nivel Silo 3',	'Peso Silo 3']

df = df.drop(drop_raw, axis=1)

df.style.format({'Animais alojados ativos': '{:0.f}',
                 'Nivel Silo 1': '{:2.f}',
                 'Peso Silo 1': '{:2.f}',
                 'Nivel Silo 2': '{:2.f}',
                 'Peso Silo 2': '{:2.f}',
                 'delta abastecimento': '{:0.f}'
                 })

df.head()

,cod_prod,Granja,Lat,Long,Município,Produção,Tipo,barn,Sexo,Peso de abate,Data alojamento,Data previsão,Peso inicial,Idade do lote,Status do lote,Animais alojados,Animais alojados ativos,Data carregamento ativos,Peso do dia,Peso esperado,farm_code,batch,Company,delta abastecimento,Nivel Silo 1,Nivel Silo 2,Peso Silo 1,Peso Silo 2,route_id,Distância (km)
28,15,Vinícius Macedo,-27.128580,-49.889523,Rio do Oeste,Suínos,Corte,111,Mixed,125.0,23/10/2024,19/02/2025,21.573,43,ativo,1530,1525,19/02/2025,59.550,57.038,52,717,4,0.0,102.24,10.94,14729.48,1576.51,15.0,27.6
36,4,Marcelo Azevedo,-26.977866,-49.873241,Dona Emma,Suínos,Corte,91,Male,125.0,23/08/2024,15/12/2024,24.700,104,ativo,1560,1524,15/12/2024,116.200,123.343,38,706,4,0.0,99.97,61.99,16099.38,9117.22,4.0,50.7
38,5,Mariana Costa,-27.065726,-50.253992,Passo Manso,Suínos,Corte,89,Female,133.0,23/08/2024,13/12/2024,24.900,104,ativo,1300,1300,13/12/2024,125.383,118.276,35,711,4,2.0,32.26,39.85,3874.30,4784.87,5.0,89.5
45,7,Patrícia Moreira,-27.329972,-49.845078,Trombudo Central,Suínos,Corte,83,Mixed,133.0,22/08/2024,12/12/2024,26.360,105,ativo,1481,1481,12/12/2024,126.780,123.196,32,704,4,3.0,0.86,98.34,120.95,13833.09,7.0,27.1
50,10,Renata Pires,-26.883043,-50.076173,Rio do Campo,Suínos,Corte,104,Female,133.0,22/08/2024,12/12/2024,25.010,105,ativo,1620,1620,12/12/2024,126.690,119.171,46,713,4,2.0,78.36,NaN,10935.38,NaN,10.0,99.3


In [125]:
dic_tipo_racao = {
    (0, 14): 'Crescimento Leitão',
    (15, 38):'Crescimento I',
    (39, 47):'Crescimento Reforço',
    (48, 71):'Crescimento II',
    (72, 84):'Terminação I',
    (85, 101):'Terminação II',
    (102, 202):'Terminação III',
}

dic_valor_racao ={
    'Crescimento Leitão': 1.5098,
    'Crescimento I': 1.3218,
    'Crescimento Reforço': 1.3948,
    'Crescimento II': 1.2675,
    'Terminação I': 1.4185,
    'Terminação II': 1.2349,
    'Terminação III': 1.2144,
    }

def obter_tipo_racao(dia_lote):
    for intervalo, tipo_racao in dic_tipo_racao.items():
        if intervalo[0] <= dia_lote <= intervalo[1]:
            return tipo_racao
    return None

def calcular_silo_peso(row):
    if pd.notnull(row['Peso Silo 1']) and pd.notnull(row['Peso Silo 2']):
        return (row['Peso Silo 1'] + row['Peso Silo 2']) / 2
    elif pd.notnull(row['Peso Silo 1']):
        return row['Peso Silo 1']
    elif pd.notnull(row['Peso Silo 2']):
        return row['Peso Silo 2']
    else:
        return None


def calcular_silo_nível(row):
    if pd.notnull(row['Nivel Silo 1']) and pd.notnull(row['Nivel Silo 2']):
        return (row['Nivel Silo 1'] + row['Nivel Silo 2']) / 2
    elif pd.notnull(row['Nivel Silo 1']):
        return row['Nivel Silo 1']
    elif pd.notnull(row['Nivel Silo 2']):
        return row['Nivel Silo 2']
    else:
        return None


def calcular_silo(df):
    df['Peso Silo'] = df[['Peso Silo 1', 'Peso Silo 2']].sum(axis=1, skipna=True)

    df['Peso Silo 1 Total'] = df['Peso Silo 1'] * 100 / df['Nivel Silo 1']
    df['Peso Silo 2 Total'] = df['Peso Silo 2'] * 100 / df['Nivel Silo 2']

    df['Peso Silo 1 Total'] = df['Peso Silo 1 Total'].fillna(0)
    df['Peso Silo 2 Total'] = df['Peso Silo 2 Total'].fillna(0)

    df['Capacidade do Silo'] = df['Peso Silo 1 Total'] + df['Peso Silo 2 Total']

    df['Nivel Silo'] = df['Peso Silo'] * 100  / df['Capacidade do Silo']

    df = df.drop(columns=['Peso Silo 1', 'Peso Silo 2', 'Peso Silo 1 Total', 'Peso Silo 2 Total', 'Nivel Silo 1', 'Nivel Silo 2'])

    return df

In [126]:
df['Peso Silo'] = df.apply(calcular_silo_peso, axis=1)
df['Nivel Silo'] = df.apply(calcular_silo_nível, axis=1)

df = calcular_silo(df)

df.head()

,cod_prod,Granja,Lat,Long,Município,Produção,Tipo,barn,Sexo,Peso de abate,Data alojamento,Data previsão,Peso inicial,Idade do lote,Status do lote,Animais alojados,Animais alojados ativos,Data carregamento ativos,Peso do dia,Peso esperado,farm_code,batch,Company,delta abastecimento,route_id,Distância (km),Peso Silo,Nivel Silo,Capacidade do Silo
28,15,Vinícius Macedo,-27.128580,-49.889523,Rio do Oeste,Suínos,Corte,111,Mixed,125.0,23/10/2024,19/02/2025,21.573,43,ativo,1530,1525,19/02/2025,59.550,57.038,52,717,4,0.0,15.0,27.6,16305.99,56.584070,28817.280271
36,4,Marcelo Azevedo,-26.977866,-49.873241,Dona Emma,Suínos,Corte,91,Male,125.0,23/08/2024,15/12/2024,24.700,104,ativo,1560,1524,15/12/2024,116.200,123.343,38,706,4,0.0,4.0,50.7,25216.60,81.840784,30811.777000
38,5,Mariana Costa,-27.065726,-50.253992,Passo Manso,Suínos,Corte,89,Female,133.0,23/08/2024,13/12/2024,24.900,104,ativo,1300,1300,13/12/2024,125.383,118.276,35,711,4,2.0,5.0,89.5,8659.17,36.054620,24016.811431
45,7,Patrícia Moreira,-27.329972,-49.845078,Trombudo Central,Suínos,Corte,83,Mixed,133.0,22/08/2024,12/12/2024,26.360,105,ativo,1481,1481,12/12/2024,126.780,123.196,32,704,4,3.0,7.0,27.1,13954.04,49.604578,28130.548973
50,10,Renata Pires,-26.883043,-50.076173,Rio do Campo,Suínos,Corte,104,Female,133.0,22/08/2024,12/12/2024,25.010,105,ativo,1620,1620,12/12/2024,126.690,119.171,46,713,4,2.0,10.0,99.3,10935.38,78.360000,13955.308831


In [127]:
drop_raw_priority = ['Tipo', 'barn',	'Sexo',	'Peso de abate',	'Data alojamento',	'Data previsão',	'Peso inicial',
                     'Status do lote',	'Animais alojados', 'Peso do dia',	'Peso esperado', 'farm_code', 'batch', 'Company']

priority_delivery = df.sort_values(['delta abastecimento'], ascending = True).drop(drop_raw_priority, axis=1)

priority_delivery.rename(columns={'Idade do lote': 'Tipo Ração',
                                  'delta abastecimento': "Dias Restantes",
                                  'Animais alojados ativos': 'Qtd Animais',
                                  'Data carregamento ativos': 'Previsão do Abate',
                                  'Peso Total': 'Capacidade do Silo',
                                  'Distância (km)': 'Distância - ida (km)',
                                  }, inplace=True)

priority_delivery['Tipo_racao'] = priority_delivery['Tipo Ração'].apply(obter_tipo_racao)

priority_delivery = priority_delivery.sort_values(by=['Dias Restantes'], ascending = True).reset_index(drop=True)

In [128]:
priority_delivery['Consumo/Dia Médio'] = priority_delivery['Tipo Ração'].map(df_racoes.set_index('Dia_LOTE')['Consumo Diário (kg/cabeça)'])
priority_delivery['Consumo/Dia Médio'] = priority_delivery['Consumo/Dia Médio'] * priority_delivery['Qtd Animais']

priority_delivery['A Enviar'] = priority_delivery['Capacidade do Silo'] - priority_delivery['Consumo/Dia Médio']
priority_delivery['Duração da Ração'] = priority_delivery['Capacidade do Silo'] / priority_delivery['Consumo/Dia Médio']

priority_delivery['Data de Entrega'] = datetime.now() + pd.to_timedelta(priority_delivery['Dias Restantes'], unit='D') - timedelta(days=1)
priority_delivery['Data Final da Ração'] = priority_delivery['Data de Entrega'] + pd.to_timedelta(priority_delivery['Duração da Ração'], unit='D')

priority_delivery['Data de Entrega Original'] = priority_delivery['Data de Entrega']
priority_delivery['Data Final da Ração Original'] = priority_delivery['Data Final da Ração']

data_referencia = pd.to_datetime('today').normalize()
priority_delivery['Data de Entrega'] = (priority_delivery['Data de Entrega'] - data_referencia).dt.days
priority_delivery['Data Final da Ração'] = (priority_delivery['Data Final da Ração'] - data_referencia).dt.days

In [129]:
df_racoes = df_racoes.sort_values(by='Dia_LOTE')

racoes = {}
intervalo_inicio = None
tipo_racao_anterior = None

for i, row in df_racoes.iterrows():
    dia = row['Dia_LOTE']
    tipo_racao = row['Fase']

    if tipo_racao_anterior is None:
        intervalo_inicio = dia
        tipo_racao_anterior = tipo_racao

    if tipo_racao != tipo_racao_anterior:
        racoes[(intervalo_inicio, dia - 1)] = tipo_racao_anterior
        intervalo_inicio = dia
        tipo_racao_anterior = tipo_racao

racoes[(intervalo_inicio, df_racoes['Dia_LOTE'].max())] = tipo_racao_anterior

In [130]:
def formatar_tempo_horas(tempo):
    horas, minutos = divmod(tempo * 60, 60)
    return f"{int(horas):02}:{int(minutos):02}"

def definir_intervalos_racao(df_racoes):
    racoes = {}
    intervalo_inicio = None
    tipo_racao_anterior = None

    df_racoes = df_racoes.sort_values(by='Dia_LOTE').reset_index(drop=True)

    for i, row in df_racoes.iterrows():
        dia = row['Dia_LOTE']
        tipo_racao = row['Fase']

        if tipo_racao_anterior is None:
            intervalo_inicio = dia
            tipo_racao_anterior = tipo_racao

        if tipo_racao != tipo_racao_anterior:
            racoes[(intervalo_inicio, dia - 1)] = tipo_racao_anterior
            intervalo_inicio = dia
            tipo_racao_anterior = tipo_racao

    racoes[(intervalo_inicio, df_racoes['Dia_LOTE'].max())] = tipo_racao_anterior

    return racoes

def obter_racao(dia, racoes):
    for intervalo, racao in racoes.items():
        if intervalo[0] <= dia <= intervalo[1]:
            return racao
    return None  # Retorna None se o dia não estiver em nenhum intervalo


def calcular_quantidades(data_inicio, data_fim, quantidade, racoes):
    racoes_quantidade = {}
    dias = pd.date_range(start=data_inicio, end=data_fim)
    total_dias = len(dias)
    dias_por_racao = {}

    for dia in dias:
        delta_dias = (dia - pd.to_datetime('today')).days
        racao = obter_racao(delta_dias, racoes)

        if racao:
            if racao in dias_por_racao:
                dias_por_racao[racao] += 1
            else:
                dias_por_racao[racao] = 1

    for racao, dias in dias_por_racao.items():
        racoes_quantidade[racao] = quantidade * (dias / total_dias)

    return racoes_quantidade


def expandir_racoes(racoes):
    racao_items = list(racoes.items())
    while len(racao_items) < 2:
      racao_items.append((None, None))
    result = []
    for item in racao_items:
        result.extend(item)
    return result[:4]


def calcular_total_racoes(df):
  total_racoes = {}
  for _, row in df.iterrows():
      for i in range(1, 3):
          tipo_racao = row[f'Tipo Ração {i}']
          quantidade_racao = row[f'Quantidade Ração {i}']
          if tipo_racao and not pd.isna(quantidade_racao):
              if tipo_racao in total_racoes:
                  total_racoes[tipo_racao] += float(quantidade_racao)
              else:
                  total_racoes[tipo_racao] = float(quantidade_racao)

In [131]:
def calcular_tipo_e_quantidade_racao(df, df_racoes):
    df['Idade Inicial'] = df['Tipo Ração'] + df['Dias Restantes'] - 1
    df['Idade Final'] = df['Tipo Ração'] + df['Data Final da Ração']

    intervalos_racao = definir_intervalos_racao(df_racoes)

    racao_tipo_1 = []
    quantidade_1 = []
    racao_tipo_2 = []
    quantidade_2 = []

    def obter_racao_por_idade(idade, intervalos):
        for (inicio, fim), tipo in intervalos.items():
            if inicio <= idade <= fim:
                return tipo
        return None

    for idx, row in df.iterrows():
        idade_inicial = row['Idade Inicial']
        idade_final = row['Idade Final']

        racao_inicio = obter_racao_por_idade(idade_inicial, intervalos_racao)
        racao_fim = obter_racao_por_idade(idade_final, intervalos_racao)

        if racao_inicio == racao_fim:
            racao_tipo_1.append(racao_inicio)
            quantidade_1.append(row['A Enviar'])
            racao_tipo_2.append(None)
            quantidade_2.append(None)
        else:
            dias_racao_inicio = next((fim for (inicio, fim), tipo in intervalos_racao.items() if tipo == racao_inicio), 0) - idade_inicial + 1
            dias_racao_fim = idade_final - next((inicio for (inicio, fim), tipo in intervalos_racao.items() if tipo == racao_fim), 0) + 1

            total_dias = dias_racao_inicio + dias_racao_fim
            quantidade_inicio = (dias_racao_inicio / total_dias) * row['A Enviar']
            quantidade_fim = (dias_racao_fim / total_dias) * row['A Enviar']

            racao_tipo_1.append(racao_inicio)
            quantidade_1.append(quantidade_inicio)
            racao_tipo_2.append(racao_fim)
            quantidade_2.append(quantidade_fim)

    df['Ração Tipo 1'] = racao_tipo_1
    df['Quantidade 1'] = quantidade_1
    df['Ração Tipo 2'] = racao_tipo_2
    df['Quantidade 2'] = quantidade_2

    return df

In [132]:
calcular_tipo_e_quantidade_racao(priority_delivery, df_racoes)

priority_delivery.head()

,cod_prod,Granja,Lat,Long,Município,Produção,Tipo Ração,Qtd Animais,Previsão do Abate,Dias Restantes,route_id,Distância - ida (km),Peso Silo,Nivel Silo,Capacidade do Silo,Tipo_racao,Consumo/Dia Médio,A Enviar,Duração da Ração,Data de Entrega,Data Final da Ração,Data de Entrega Original,Data Final da Ração Original,Idade Inicial,Idade Final,Ração Tipo 1,Quantidade 1,Ração Tipo 2,Quantidade 2
0,15,Vinícius Macedo,-27.128580,-49.889523,Rio do Oeste,Suínos,43,1525,19/02/2025,0.0,15.0,27.6,16305.99,56.584070,28817.280271,Crescimento Reforço,3584.610541,25232.669730,8.039166,-1,7,2025-11-24 15:31:41.215259,2025-12-02 16:28:05.183903993,42.0,50,Crescimento reforço,16821.779820,Crescimento II,8410.889910
1,4,Marcelo Azevedo,-26.977866,-49.873241,Dona Emma,Suínos,104,1524,15/12/2024,0.0,4.0,50.7,25216.60,81.840784,30811.777000,Terminação III,4452.672198,26359.104802,6.919840,-1,6,2025-11-24 15:31:41.215259,2025-12-01 13:36:15.348969448,103.0,110,Terminação III,26359.104802,None,NaN
2,13,Tiago Pereira,-27.488242,-49.632625,Petrolândia,Suínos,100,1195,04/12/2024,0.0,13.0,48.5,3117.25,26.850000,11609.869646,Terminação II,3488.537905,8121.331741,3.328004,-1,2,2025-11-24 15:31:41.215259,2025-11-27 23:24:00.799063317,99.0,102,Terminação II,6090.998806,Terminação III,2030.332935
3,1,Leonardo Farias,-27.067259,-49.834198,Mirador,Suínos,43,1349,18/02/2025,0.0,1.0,34.2,20940.45,90.114670,23237.559400,Crescimento Reforço,3170.911226,20066.648174,7.328354,-1,6,2025-11-24 15:31:41.215259,2025-12-01 23:24:30.985452549,42.0,49,Crescimento reforço,15049.986131,Crescimento II,5016.662044
4,6,Natália Soares,-27.045991,-50.234870,Passo Manso,Suínos,56,1287,04/02/2025,0.0,6.0,86.9,18072.64,58.983357,30640.236457,Crescimento II,3185.866899,27454.369558,9.617551,-1,9,2025-11-24 15:31:41.215259,2025-12-04 06:20:57.596595146,55.0,65,Crescimento II,27454.369558,None,NaN


In [133]:
def dividir_em_gavetas(df):
    df['Quantidade 1'] = df['Quantidade 1'].fillna(0)
    df['Quantidade 2'] = df['Quantidade 2'].fillna(0)

    df['Gavetas Ração 1'] = np.floor(df['Quantidade 1'] / 3000).astype(int)
    df['Quantidade Ração 1'] = df['Gavetas Ração 1'] * 3000
    df['Sobra Ração 1'] = df['Quantidade 1'] - (df['Gavetas Ração 1'] * 3000)

    df['Gavetas Ração 2'] = np.floor(df['Quantidade 2'] / 3000).astype(int)
    df['Quantidade Ração 2'] = df['Gavetas Ração 2'] * 3000
    df['Sobra Ração 2'] = df['Quantidade 2'] - (df['Gavetas Ração 2'] * 3000)

    df['Gavetas'] = df.apply(lambda row: (row['Gavetas Ração 1'], row['Gavetas Ração 2']), axis=1)
    df['Quantidade Total Arredondada'] = df['Quantidade Ração 1'] + df['Quantidade Ração 2']

    return df

In [134]:
priority_delivery['Dias Restantes'] = pd.to_numeric(priority_delivery['Dias Restantes'], errors='coerce')
priority_delivery = priority_delivery.sort_values(['Dias Restantes', 'Nivel Silo'], ascending = True)

priority_delivery = priority_delivery.reset_index(drop=True)
priority_delivery.index = priority_delivery.index + 1

In [ ]:
priority_delivery['Distância - ida (km)'] = priority_delivery['Distância - ida (km)'].astype(str)
priority_delivery['Distância - ida (km)'] = priority_delivery['Distância - ida (km)'].str.replace(',', '.')
priority_delivery['Distância - ida (km)'] = pd.to_numeric(priority_delivery['Distância - ida (km)'], errors='coerce')

priority_delivery['Distância - ida e volta (km)'] = priority_delivery['Distância - ida (km)'] * 2

colunas = list(priority_delivery.columns)
colunas.insert(-1, colunas.pop(colunas.index('Distância - ida (km)')))
priority_delivery = priority_delivery[colunas]

priority_delivery['Tempo (h) - ida'] = priority_delivery['Distância - ida (km)'] / 35
priority_delivery['Tempo (h) - ida e volta'] = priority_delivery['Tempo (h) - ida'] * 2

priority_delivery['Tempo (h) - ida'] = priority_delivery['Tempo (h) - ida'].apply(formatar_tempo_horas)
priority_delivery['Tempo (h) - ida e volta'] = priority_delivery['Tempo (h) - ida e volta'].apply(formatar_tempo_horas)


priority_delivery['Consumo de Combustível - ida (l)'] = priority_delivery['Distância - ida (km)'] / 2.5
priority_delivery['Consumo de Combustível - ida e volta (l)'] = priority_delivery['Consumo de Combustível - ida (l)'] * 2

priority_delivery['Custo de Deslocamento - ida (R$)'] = priority_delivery['Consumo de Combustível - ida (l)'] * 6.00
priority_delivery['Custo de Deslocamento - ida e volta (R$)'] = priority_delivery['Custo de Deslocamento - ida (R$)'] * 2

priority_delivery['A Enviar'] = pd.to_numeric(priority_delivery['A Enviar'], errors='coerce')
priority_delivery['Custo Por kg Enviado - ida (R$/kg/km)'] = priority_delivery['Custo de Deslocamento - ida (R$)'] / priority_delivery['A Enviar']
priority_delivery['Custo Por kg Enviado - ida e volta (R$/kg/km)'] = priority_delivery['Custo Por kg Enviado - ida (R$/kg/km)'] * 2

priority_delivery['CO2 - ida'] = priority_delivery['Consumo de Combustível - ida (l)'] * 3.2
priority_delivery['CO2 - ida e volta'] = priority_delivery['CO2 - ida'] * 2

priority_delivery['Dias Restantes'] = priority_delivery['Dias Restantes'].astype(int)
priority_delivery['route_id'] = priority_delivery['route_id'].astype(int)

In [136]:
priority_delivery['Data de Entrega'] = priority_delivery['Data de Entrega Original'].dt.strftime('%d/%m/%Y')
priority_delivery['Data Final da Ração'] = priority_delivery['Data Final da Ração Original'].dt.strftime('%d/%m/%Y')

priority_delivery['Previsão do Abate'] = pd.to_datetime(priority_delivery['Previsão do Abate'], format='%d/%m/%Y')
priority_delivery['Previsão do Abate'] = priority_delivery['Previsão do Abate'].dt.strftime('%d/%m/%Y')

priority_delivery = priority_delivery.drop(columns=['Data de Entrega Original', 'Data Final da Ração Original', 'Tipo Ração'])

In [137]:
priority_delivery['Data de Entrega'] = pd.to_datetime(priority_delivery['Data de Entrega'], format='%d/%m/%Y')
priority_delivery['Data Final da Ração'] = pd.to_datetime(priority_delivery['Data Final da Ração'], format='%d/%m/%Y')
priority_delivery['Previsão do Abate'] = pd.to_datetime(priority_delivery['Previsão do Abate'], format='%d/%m/%Y')

data_ref_antiga = priority_delivery["Data de Entrega"].min()
data_ref_nova = pd.Timestamp.today().normalize()

priority_delivery["dif_Data de Entrega"] = priority_delivery["Data de Entrega"] - data_ref_antiga
priority_delivery["dif_Data Final da Ração"] = priority_delivery["Data Final da Ração"] - data_ref_antiga
priority_delivery["dif_Previsão do Abate"] = priority_delivery["Previsão do Abate"] - data_ref_antiga

priority_delivery["Data de Entrega"] = data_ref_nova + priority_delivery["dif_Data de Entrega"]
priority_delivery["Data Final da Ração"] = data_ref_nova + priority_delivery["dif_Data Final da Ração"]
priority_delivery["Previsão do Abate"] = data_ref_nova + priority_delivery["dif_Previsão do Abate"]

priority_delivery = priority_delivery.drop(columns=["dif_Data de Entrega", "dif_Data Final da Ração", "dif_Previsão do Abate"])

In [138]:
dividir_em_gavetas(priority_delivery)

,cod_prod,Granja,Lat,Long,Município,Produção,Qtd Animais,Previsão do Abate,Dias Restantes,route_id,Peso Silo,Nivel Silo,Capacidade do Silo,Tipo_racao,Consumo/Dia Médio,A Enviar,Duração da Ração,Data de Entrega,Data Final da Ração,Idade Inicial,Idade Final,Ração Tipo 1,Quantidade 1,Ração Tipo 2,Quantidade 2,Distância - ida (km),Distância - ida e volta (km),Tempo (h) - ida,Tempo (h) - ida e volta,Consumo de Combustível - ida (l),Consumo de Combustível - ida e volta (l),Custo de Deslocamento - ida (R$),Custo de Deslocamento - ida e volta (R$),Custo Por kg Enviado - ida (R$/kg/km),Custo Por kg Enviado - ida e volta (R$/kg/km),CO2 - ida,CO2 - ida e volta,Gavetas Ração 1,Quantidade Ração 1,Sobra Ração 1,Gavetas Ração 2,Quantidade Ração 2,Sobra Ração 2,Gavetas,Quantidade Total Arredondada
1,8,Priscila Nogueira,-26.884201,-50.071748,Rio do Campo,Suínos,1285,2025-02-10,0,8,7729.74,24.006114,32199.047459,Crescimento II,3087.433024,29111.614435,10.429068,2025-11-25,2025-12-06,47.0,58,Crescimento reforço,2425.967870,Crescimento II,26685.646566,101.0,202.0,02:53,05:46,40.40,80.80,242.40,484.80,0.008327,0.016653,129.280,258.560,0,0,2425.967870,8,24000,2685.646566,"(0, 8)",24000
2,13,Tiago Pereira,-27.488242,-49.632625,Petrolândia,Suínos,1195,2024-12-05,0,13,3117.25,26.850000,11609.869646,Terminação II,3488.537905,8121.331741,3.328004,2025-11-25,2025-11-28,99.0,102,Terminação II,6090.998806,Terminação III,2030.332935,48.5,97.0,01:23,02:46,19.40,38.80,116.40,232.80,0.014333,0.028665,62.080,124.160,2,6000,90.998806,0,0,2030.332935,"(2, 0)",6000
3,15,Vinícius Macedo,-27.128580,-49.889523,Rio do Oeste,Suínos,1525,2025-02-20,0,15,16305.99,56.584070,28817.280271,Crescimento Reforço,3584.610541,25232.669730,8.039166,2025-11-25,2025-12-03,42.0,50,Crescimento reforço,16821.779820,Crescimento II,8410.889910,27.6,55.2,00:47,01:34,11.04,22.08,66.24,132.48,0.002625,0.005250,35.328,70.656,5,15000,1821.779820,2,6000,2410.889910,"(5, 2)",21000
4,6,Natália Soares,-27.045991,-50.234870,Passo Manso,Suínos,1287,2025-02-05,0,6,18072.64,58.983357,30640.236457,Crescimento II,3185.866899,27454.369558,9.617551,2025-11-25,2025-12-05,55.0,65,Crescimento II,27454.369558,None,0.000000,86.9,173.8,02:28,04:57,34.76,69.52,208.56,417.12,0.007597,0.015193,111.232,222.464,9,27000,454.369558,0,0,0.000000,"(9, 0)",27000
5,12,Tatiane Cardoso,-26.853437,-49.848190,Vitor Meirelles,Suínos,1184,2025-01-30,0,12,17451.48,77.169206,22614.564630,Crescimento II,2950.013753,19664.550876,7.665918,2025-11-25,2025-12-03,57.0,65,Crescimento II,19664.550876,None,0.000000,81.8,163.6,02:20,04:40,32.72,65.44,196.32,392.64,0.009983,0.019967,104.704,209.408,6,18000,1664.550876,0,0,0.000000,"(6, 0)",18000
6,4,Marcelo Azevedo,-26.977866,-49.873241,Dona Emma,Suínos,1524,2024-12-16,0,4,25216.60,81.840784,30811.777000,Terminação III,4452.672198,26359.104802,6.919840,2025-11-25,2025-12-02,103.0,110,Terminação III,26359.104802,None,0.000000,50.7,101.4,01:26,02:53,20.28,40.56,121.68,243.36,0.004616,0.009232,64.896,129.792,8,24000,2359.104802,0,0,0.000000,"(8, 0)",24000
7,1,Leonardo Farias,-27.067259,-49.834198,Mirador,Suínos,1349,2025-02-19,0,1,20940.45,90.114670,23237.559400,Crescimento Reforço,3170.911226,20066.648174,7.328354,2025-11-25,2025-12-02,42.0,49,Crescimento reforço,15049.986131,Crescimento II,5016.662044,34.2,68.4,00:58,01:57,13.68,27.36,82.08,164.16,0.004090,0.008181,43.776,87.552,5,15000,49.986131,1,3000,2016.662044,"(5, 1)",18000
8,5,Mariana Costa,-27.065726,-50.253992,Passo Manso,Suínos,1300,2024-12-14,2,5,8659.17,36.054620,24016.811431,Terminação III,3798.211192,20218.600239,6.323190,2025-11-27,2025-12-03,105.0,111,Terminação III,20218.600239,None,0.000000,89.5,179.0,02:33,05:06,35.80,71.60,214.80,429.60,0.010624,0.021248,114.560,229.120,6,18000,2218.600239,0,0,0.000000,"(6, 0)",18000
9,10,Renata Pires,-26.883043,-50.076173,Rio do Campo,Suínos,1620,2024-12-13,2,10,10935.38,78.360000,13955.308831,Terminação III,4731.639096,9223.669735,2.949360,2025-11-27,2025-11-30,106.0,109,Terminação III

In [139]:
priority_delivery.to_csv("priority_delivery.csv", index=False)